In [22]:
!pip3 install pandas numpy scikit-learn yfinance alpha_vantage requests beautifulsoup4 vaderSentiment quandl


alpha ZQOPKB408D1QXCA4
newsAPI 19e9cf65fc224b5ca79d3932084b982d

In [7]:
import requests
import pandas as pd

# Alpha Vantage API key
api_key = 'ZQOPKB408D1QXCA4'  # Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key

# Function to fetch earnings data from Alpha Vantage API
def fetch_earnings_data(ticker):
    url = f'https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={api_key}'
    response = requests.get(url)
    data = response.json()
    
    # Check if the response contains earnings data
    if "annualEarnings" in data and "quarterlyEarnings" in data:
        # Process annual earnings
        annual_earnings = pd.DataFrame(data["annualEarnings"])
        annual_earnings['Ticker'] = ticker  # Add ticker column for reference
        
        # Process quarterly earnings
        quarterly_earnings = pd.DataFrame(data["quarterlyEarnings"])
        quarterly_earnings['Ticker'] = ticker  # Add ticker column for reference

        return annual_earnings, quarterly_earnings
    else:
        print(f"No earnings data found for {ticker}.")
        return pd.DataFrame(), pd.DataFrame()

# Example usage
tickers = ['AAPL', 'MSFT', 'GOOGL']  # Replace with desired tickers
earnings_data = {}

for ticker in tickers:
    annual_df, quarterly_df = fetch_earnings_data(ticker)
    earnings_data[ticker] = {'Annual': annual_df, 'Quarterly': quarterly_df}

    # Save data to CSV files
    if not annual_df.empty:
        annual_df.to_csv(f'{ticker}_annual_earnings.csv', index=False)
    if not quarterly_df.empty:
        quarterly_df.to_csv(f'{ticker}_quarterly_earnings.csv', index=False)


In [ ]:
import yfinance as yf
import pandas as pd

# Define a function to download earnings data
def download_earnings_data(tickers, start_date, end_date):
    data = []
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        hist = stock.history(start=start_date, end=end_date, interval='1d')
        
        # Get income statement for net income
        income_stmt = stock.income_stmt
        
        # Extract relevant information from income statement
        if income_stmt is not None:
            net_income = income_stmt.loc['Net Income']
            # Calculate earnings growth as a percentage
            earnings_growth = net_income.pct_change().dropna() * 100
        else:
            net_income = None
            earnings_growth = None
        
        # Append data
        data.append({
            'Company': ticker,
            'Net Income': net_income,
            'Earnings Growth (%)': earnings_growth
        })
    
    return pd.DataFrame(data)

# Example usage
tickers = ['AAPL', 'MSFT', 'GOOGL']
earnings_df = download_earnings_data(tickers, '2015-01-01', '2024-01-01')
earnings_df.to_csv('earnings_data.csv', index=False)


/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_60738/2708161713.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  earnings_growth = net_income.pct_change().dropna() * 100
/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_60738/2708161713.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  earnings_growth = net_income.pct_change().dropna() * 100
/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_60738/2708161713.py:18: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a 

In [18]:
import requests
import pandas as pd

# News API key
api_key = '19e9cf65fc224b5ca79d3932084b982d'

# Function to fetch news data
def fetch_news_data(company):
    url = f"https://newsapi.org/v2/everything?q={company}&apiKey={api_key}"
    response = requests.get(url)
    articles = response.json().get('articles', [])
    
    # Extract relevant data
    data = []
    for article in articles:
        data.append({
            'Company': company,
            'Date': article['publishedAt'],
            'News Headline': article['title'],
            'Source': article['source']['name'],
            'URL': article['url']
        })
    
    return pd.DataFrame(data)

# Example usage
companies = ['Apple', 'Microsoft', 'Google']
news_data = pd.concat([fetch_news_data(company) for company in companies], ignore_index=True)

# Save to CSV
news_data.to_csv('news_sentiment.csv', index=False)


In [19]:
import yfinance as yf
import pandas as pd

def fetch_supply_demand_data(ticker, start_date, end_date):
    stock = yf.Ticker(ticker)
    hist = stock.history(start=start_date, end=end_date, interval='1d')
    
    # Extract relevant columns
    data = hist[['Volume', 'Close']]
    data['Ticker'] = ticker
    
    return data

# Example usage
tickers = ['AAPL', 'MSFT', 'GOOGL']
supply_demand_data = pd.concat([fetch_supply_demand_data(ticker, '2023-01-01', '2024-01-01') for ticker in tickers])

# Save to CSV
supply_demand_data.to_csv('supply_demand_data.csv', index=False)


/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_60738/981840227.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ticker'] = ticker
/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_60738/981840227.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Ticker'] = ticker
/var/folders/83/3t5wk1pn2fq83ss49zg61zf40000gn/T/ipykernel_60738/981840227.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

In [3]:
import requests
import pandas as pd

# FMP API key
api_key = 'lhjT6XFmqVkPV78AlKyuNtip6aymeVgT'

# Function to fetch institutional ownership data
def fetch_institutional_ownership(ticker):
    url = f"https://financialmodelingprep.com/api/v4/institutional-ownership/name?name={Apple}&apikey={api_key}"
    response = requests.get(url)
    data = response.json()
    
    # Check if data is a list (valid response) or not
    if isinstance(data, list) and len(data) > 0:
        # Convert list of dictionaries to DataFrame
        df = pd.DataFrame(data)
        return df
    else:
        print(f"No institutional ownership data found for {ticker}.")
        # Return an empty DataFrame if data is not valid
        return pd.DataFrame()

# Example usage
tickers = ['AAPL', 'MSFT', 'GOOGL']
institutional_data_list = []

# Fetch data for each ticker and store in a list
for ticker in tickers:
    df = fetch_institutional_ownership(ticker)
    if not df.empty:
        institutional_data_list.append(df)

# Concatenate all data into a single DataFrame
if institutional_data_list:
    institutional_data = pd.concat(institutional_data_list, ignore_index=True)
    # Save to CSV
    institutional_data.to_csv('institutional_ownership.csv', index=False)
else:
    print("No institutional data to save.")

NameError: name 'Apple' is not defined

In [24]:
import quandl
import pandas as pd

# Quandl API key
quandl.ApiConfig.api_key = 'WMEE_cWEipLGRy6eaowK'

# Function to fetch economic indicators data
def fetch_economic_indicators(indicator_code):
    data = quandl.get(indicator_code)
    return data

# Example usage
indicators = ['FRED/GDP', 'FRED/UNRATE', 'FRED/CPIAUCSL']
economic_data = pd.concat([fetch_economic_indicators(indicator) for indicator in indicators], axis=1)

# Save to CSV
economic_data.to_csv('market_direction.csv')


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd

analyzer = SentimentIntensityAnalyzer()

# Sample data for sentiment analysis
data = {'Company': ['AAPL', 'MSFT'], 'News Headline': ['Apple releases new iPhone', 'Microsoft acquires new AI startup']}

news_df = pd.DataFrame(data)

# Calculate sentiment scores
news_df['Sentiment Score'] = news_df['News Headline'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

news_df.to_csv('news_sentiment.csv', index=False)


In [ ]:
# Read all CSV files
earnings_df = pd.read_csv('earnings_data.csv')
news_df = pd.read_csv('news_sentiment.csv')
supply_demand_df = pd.read_csv('supply_demand_data.csv')
leadership_df = pd.read_csv('market_leadership.csv')
ownership_df = pd.read_csv('institutional_ownership.csv')
market_df = pd.read_csv('market_direction.csv')

# Merge all DataFrames
merged_df = earnings_df.merge(news_df, on='Company').merge(
    supply_demand_df, on=['Company', 'Date']
).merge(
    leadership_df, on=['Company']
).merge(
    ownership_df, on=['Company', 'Date']
).merge(
    market_df, on='Date'
)

# Save merged data
merged_df.to_csv('merged_CANSLIM_data.csv', index=False)


FileNotFoundError: [Errno 2] No such file or directory: 'supply_demand_data.csv'

# re

In [8]:
import pandas as pd
import yfinance as yf
from alpha_vantage.fundamentaldata import FundamentalData
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def collect_canslim_data(tickers, start_date, end_date, alpha_vantage_key):
    fd = FundamentalData(key=alpha_vantage_key)
    analyzer = SentimentIntensityAnalyzer()
    all_data = []

    for ticker in tickers:
        # Fetch stock data
        stock = yf.Ticker(ticker)
        hist_data = stock.history(start=start_date, end=end_date)
        
        # Fetch income statement
        income_stmt = fd.get_income_statement_annual(ticker)[0]
        
        # Calculate earnings growth
        net_income = pd.to_numeric(income_stmt.loc['netIncome'], errors='coerce')
        earnings_growth = net_income.pct_change().dropna() * 100
        
        # Fetch news data (you might want to replace this with a real news API)
        news_headlines = stock.news
        
        for date, row in hist_data.iterrows():
            data = {
                'Date': date,
                'Ticker': ticker,
                'Close': row['Close'],
                'Volume': row['Volume'],
                'EPS': stock.info.get('trailingEPS', None),
                'Earnings_Growth': earnings_growth.iloc[-1] if not earnings_growth.empty else None,
            }
            
            # Add sentiment analysis
            if news_headlines:
                headlines = [news['title'] for news in news_headlines if date.date() == pd.to_datetime(news['providerPublishTime'], unit='s').date()]
                if headlines:
                    sentiment = analyzer.polarity_scores(' '.join(headlines))['compound']
                    data['Sentiment'] = sentiment
            
            all_data.append(data)
    
    return pd.DataFrame(all_data)

# Example usage
tickers = ['AAPL', 'MSFT', 'GOOGL']
start_date = '2023-01-01'
end_date = '2024-01-01'
alpha_vantage_key = 'ZQOPKB408D1QXCA4'

df = collect_canslim_data(tickers, start_date, end_date, alpha_vantage_key)
df.to_csv('canslim_data.csv', index=False)

KeyError: 'netIncome'

In [2]:
import pandas as pd
import numpy as np

def canslim_analysis(df):
    results = []
    for ticker in df['Ticker'].unique():
        stock_data = df[df['Ticker'] == ticker]
        
        # Current quarterly earnings
        current_eps = stock_data['EPS'].iloc[-1]
        
        # Annual earnings growth
        earnings_growth = stock_data['Earnings_Growth'].iloc[-1]
        
        # New products, management, highs
        new_high = stock_data['Close'].iloc[-1] >= stock_data['Close'].max() * 0.95
        
        # Supply and demand
        volume_increase = stock_data['Volume'].pct_change().mean() > 0
        
        # Leader or laggard
        industry_performance = stock_data['Close'].pct_change().mean()
        
        # Institutional sponsorship
        # This would require additional data on institutional ownership
        
        # Market direction
        market_uptrend = stock_data['Close'].pct_change().mean() > 0
        
        score = sum([
            current_eps > 0,
            earnings_growth > 25,
            new_high,
            volume_increase,
            industry_performance > 0,
            market_uptrend
        ])
        
        results.append({
            'Ticker': ticker,
            'CANSLIM_Score': score,
            'Current_EPS': current_eps,
            'Earnings_Growth': earnings_growth,
            'New_High': new_high,
            'Volume_Increase': volume_increase,
            'Industry_Performance': industry_performance,
            'Market_Uptrend': market_uptrend
        })
    
    return pd.DataFrame(results)

# Example usage
df = pd.read_csv('canslim_data.csv')
results = canslim_analysis(df)
results.to_csv('canslim_results.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'canslim_data.csv'

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

def visualize_canslim_results(results):
    plt.figure(figsize=(12, 6))
    sns.barplot(x='Ticker', y='CANSLIM_Score', data=results)
    plt.title('CANSLIM Scores by Stock')
    plt.ylabel('CANSLIM Score')
    plt.savefig('canslim_scores.png')
    plt.close()

    plt.figure(figsize=(12, 8))
    sns.heatmap(results.set_index('Ticker').iloc[:, 1:], annot=True, cmap='YlGnBu')
    plt.title('CANSLIM Factors Heatmap')
    plt.savefig('canslim_heatmap.png')
    plt.close()

# Example usage
results = pd.read_csv('canslim_results.csv')
visualize_canslim_results(results)

FileNotFoundError: [Errno 2] No such file or directory: 'canslim_results.csv'

In [4]:
import pandas as pd
from data_collection import collect_canslim_data
from canslim_analysis import canslim_analysis
from visualization import visualize_canslim_results

def main():
    # Set up parameters
    tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB']  # Add more tickers as needed
    start_date = '2023-01-01'
    end_date = '2024-01-01'
    alpha_vantage_key = 'YOUR_ALPHA_VANTAGE_KEY'

    # Collect data
    print("Collecting data...")
    df = collect_canslim_data(tickers, start_date, end_date, alpha_vantage_key)
    df.to_csv('canslim_data.csv', index=False)
    print("Data collected and saved to canslim_data.csv")

    # Perform CANSLIM analysis
    print("Performing CANSLIM analysis...")
    results = canslim_analysis(df)
    results.to_csv('canslim_results.csv', index=False)
    print("Analysis complete and saved to canslim_results.csv")

    # Visualize results
    print("Creating visualizations...")
    visualize_canslim_results(results)
    print("Visualizations saved as canslim_scores.png and canslim_heatmap.png")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'data_collection'

# Final


In [10]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
API_KEY = 'ZQOPKB408D1QXCA4'

def fetch_data(symbol, function, **params):
    base_url = 'https://www.alphavantage.co/query'
    params.update({
        'function': function,
        'symbol': symbol,
        'apikey': API_KEY
    })
    response = requests.get(base_url, params=params)
    return response.json()

def analyze_current_earnings(symbol):
    data = fetch_data(symbol, 'INCOME_STATEMENT', datatype='json')
    annual_reports = data.get('annualReports', [])
    
    if len(annual_reports) < 2:
        return False, "Insufficient data"
    
    current_earnings = float(annual_reports[0]['netIncome'])
    previous_earnings = float(annual_reports[1]['netIncome'])
    
    growth_rate = (current_earnings - previous_earnings) / abs(previous_earnings) * 100
    
    return growth_rate >= 25, f"Earnings growth: {growth_rate:.2f}%"

def analyze_annual_earnings(symbol):
    data = fetch_data(symbol, 'INCOME_STATEMENT', datatype='json')
    annual_reports = data.get('annualReports', [])
    
    if len(annual_reports) < 3:
        return False, "Insufficient data"
    
    growth_rates = []
    for i in range(len(annual_reports) - 1):
        current = float(annual_reports[i]['netIncome'])
        previous = float(annual_reports[i+1]['netIncome'])
        growth_rate = (current - previous) / abs(previous) * 100
        growth_rates.append(growth_rate)
    
    avg_growth_rate = sum(growth_rates) / len(growth_rates)
    
    return avg_growth_rate >= 25, f"Average annual growth: {avg_growth_rate:.2f}%"

def analyze_new_product_or_management():
    # This requires manual research and cannot be easily automated
    return None, "Manual research required for new products or management changes"

def analyze_supply_and_demand(symbol):
    data = fetch_data(symbol, 'GLOBAL_QUOTE', datatype='json')
    volume = int(data['Global Quote']['06. volume'])
    
    # You may need to adjust this threshold based on the specific stock and market
    high_volume_threshold = 100000
    
    return volume > high_volume_threshold, f"Trading volume: {volume}"

def analyze_leader_or_laggard(symbol):
    # This requires comparison with industry peers and cannot be easily automated
    return None, "Manual research required for industry comparison"

def analyze_institutional_sponsorship(symbol):
    # This data is not readily available through Alpha Vantage
    return None, "Data not available through current API"

def analyze_market_direction():
    # This requires analysis of broader market indices
    return None, "Manual analysis of market direction required"

def analyze_stock(symbol):
    results = {
        'C': analyze_current_earnings(symbol),
        'A': analyze_annual_earnings(symbol),
        'N': analyze_new_product_or_management(),
        'S': analyze_supply_and_demand(symbol),
        'L': analyze_leader_or_laggard(symbol),
        'I': analyze_institutional_sponsorship(symbol),
        'M': analyze_market_direction()
    }
    
    return results

def main():
    symbol = input("Enter the stock symbol to analyze: ")
    results = analyze_stock(symbol)
    
    print(f"\nCANSLIM Analysis for {symbol}:")
    for factor, (passed, message) in results.items():
        status = "✓" if passed else "✗" if passed is not None else "?"
        print(f"{factor}: [{status}] {message}")

if __name__ == "__main__":
    main()


CANSLIM Analysis for NVDA:
C: [✓] Earnings growth: 581.32%
A: [✓] Average annual growth: 85.32%
N: [?] Manual research required for new products or management changes
S: [✓] Trading volume: 413638098
L: [?] Manual research required for industry comparison
I: [?] Data not available through current API
M: [?] Manual analysis of market direction required
